# Calculating Calories Burned During Exercise

I love bike riding. Full stop. But my first few rides where pretty short, a notable one being a 6 km ride where I felt like I was going to die. If someone asked me, I would've said RPE of a thousand... maybe? (Yeah... it is supposed to go to ten).

I used to get a lot of cramps. A lot. Sometimes I was riding feeling like a king (the slowest one and quite round at that time) and 0.05 seconds later it was as if I injected viagra into my leg muscles. Check out the pic below from Volcano. That pic was taken with about a 3 second difference.

![](./images/cramping.png)

But that is a story which I will cover some other day. 

What I am trying to learn now is how much calories I need to consume during rides to avoid bonking. Of course I need to take into account carbs, sodium, and liquids. But for today, I'll focus on calories.

The amount of calories you burn during exercise depend on many factors, including your: 
- Size (or body weight)
- Fitness
- Efficiency
- Duration of the exercise

In this exercise I am going to estimate how many calories I burn per hour, using an example of a ride with where the terrain was pretty flat and it was a somewhat constant effort.

I am going to use several different methodologies as a comparison point and will finish with the one from *Feed Zone Portables*. Strava had my heart rate, the power meter, weight, and exact speed so I would like to think it is the most accurate. By using all the other formulas I will get a feel into how precise they are.

I asked ChatGPT and Copilot for information on which are the formulas that are commonly used to estimate calories burned. Here are the formulas ChatGPT suggested:

![](./images/chatgptformulas.png)

Regarding the book, I am reading it to learn more about nutrition during rides and it has a formula that I want to test out.



## The Ride
Ticaban, which is a plain in the northern part of Costa Rica where we did 3 rounds with a motivated peloton, which meant the speed was pretty constant. 
https://www.strava.com/activities/11225188919.  

![](./images/ticaban-11225188919.png)

## The Stats
I have a powermeter and heart rate date, which means I have a decent amount of data to play with. Since it is pretty flat, it is a good point for calculations. How flat? An elevation gain of just 249 meters (817 feet) over 122 kms (76 miles).

According to Strava, I burned 2,199 calories and a total work of 2,250 kJ, with a weighted average power of 196 W and average power of 178 W.

Let's create the variables with the data:

In [47]:
# The Data

# 121.83 kms (76 miles)
distance_meters = 121830
distance_kilometers = distance_meters / 1000

# 3 hours, 30 minutes, 17 seconds is equilavent to 12617 seconds
duration_moving_time_seconds = 12617
duration_moving_time_minutes = duration_moving_time_seconds / 60

# Elevation gain in meters
elevation_gain_m = 249

# Total work in kilojoules
work_kilojoules = 2250

# Calories burned according to Strava   
calories_strava = 2199 

# Average speed in km/h
average_speed_km_hour = 38.4

# Average power in watts
average_power_watts = 178

# Weighted average power in watts
average_power_watts_weighted = 196

# Average heart rate in beats per minute
average_heart_rate_bpm = 149

# Temperature in celsius
temperature_celsius = 26

# My age
age = 46

# My weight at Ticaban
weight_ticaban_kg = 77.5 

# My current weight
weight_kg = 69

## Strava Calculation
According to Strava, I burned 2199 calories. But what if I calculated it myself using these variables?

In [48]:
def strava_calories_estimate(weight_kg, age, distance_km, moving_time_seconds, elevation_gain_m, avg_hr=None, avg_power=None, avg_speed_kmh=None):
    """
    Estimates calories burned using Strava's methodology.
    
    Parameters:
    - weight_kg: Body weight in kilograms
    - age: Age in years
    - distance_km: Distance covered in kilometers
    - moving_time_seconds: Moving time in seconds
    - elevation_gain_m: Elevation gain in meters
    - avg_hr: Average heart rate in beats per minute (optional)
    - avg_power: Weighted average power in watts (optional)
    - avg_speed_kmh: Average speed in kilometers per hour (optional)
    
    Returns:
    - Estimated calories burned
    """
    # Convert moving time to hours
    moving_time_hours = moving_time_seconds / 3600
    
    # Priority 1: Use power data if available
    if avg_power is not None:
        calories = avg_power * 3.6 * moving_time_hours
        return calories

    # Priority 2: Use heart rate data if available
    if avg_hr is not None:
        # Simple heart rate-based estimation
        # Using a basic formula: HRmax for males: 220 - age, use 70-85% effort for moderate cycling
        hr_max = 220 - age
        intensity_factor = (avg_hr / hr_max)  # Proportion of HRmax
        # Approximation: calories per minute based on intensity
        calories_per_minute = (8 + (intensity_factor * 10)) * weight_kg / 60
        calories = calories_per_minute * (moving_time_seconds / 60)
        return calories
    
    # Priority 3: Use MET values based on average speed if no heart rate or power
    if avg_speed_kmh is not None:
        if avg_speed_kmh < 10:
            met_value = 4.0  # light effort
        elif avg_speed_kmh < 13:
            met_value = 6.8  # moderate effort
        elif avg_speed_kmh < 16:
            met_value = 8.0  # vigorous effort
        else:
            met_value = 10.0  # very vigorous effort
        
        calories = met_value * weight_kg * moving_time_hours
        return calories
    
    # If no data available, return 0
    return 0

Using the formula above, I made the two calculations and got the following results which are quite close to the *2199* calories that Strava calculated:
- I burned *2245.83* calories, if I used average power
- However, if I used *weighted* average power, the calculation was slightly higher and came out at *2472.93* calories

What is weighted average power?
Weighted average power (also known as Normalized Power) adjusts for intensity changes, giving more weight to harder efforts. It better reflects the physiological cost of the ride, especially during variable efforts like intervals or races. Thus, weighted average power is a more accurate measure of the ride's true intensity and stress on the body.

In this case I had opened my mouth and asked a couple of my friends (that are waaaay better than me) to just go ahead and kill me. They _almost_ succeeded! hehe

Needless to say, this ride is quite particular because of how flat the terrain is. Going uphill will burn more calories and will be harder to estimate, but for all intents and purposes what I want is to *learn how to calculate within reasonable limits how much calories I need to take on my longer rides*.

In [51]:
# Example usage with the provided data using the formula I found on the internet for Strava
calories_burned = strava_calories_estimate(weight_ticaban_kg, age, distance_kilometers, duration_moving_time_seconds, elevation_gain_m, average_heart_rate_bpm, average_power_watts, average_speed_km_hour)
print(f"Calories burned using Strava calculation and average power: {calories_burned:.2f}")

calories_burned = strava_calories_estimate(weight_ticaban_kg, age, distance_kilometers, duration_moving_time_seconds, elevation_gain_m, average_heart_rate_bpm, average_power_watts_weighted, average_speed_km_hour)
print(f"Calories burned using Strava calculation and weighted average power: {calories_burned:.2f}")

Calories burned using Strava calculation and average power: 2245.83
Calories burned using Strava calculation and average power: 2472.93


So this formula is pretty decent for this particular ride, but what if I test it on a ride with a good elevation gain, like for example the Alto de Letras? (Which is the world's biggest climb, according to GCN: https://www.youtube.com/watch?v=WIdpuVGO5zY)

Let's try it out.

![](./images/strava_letras.png)

According to Strava I burned *4713* calories which is really close to the *4776* calories estimated using the formula above.

_Conclusion: Apparently, regarldess of whether it is a flat ride or the biggest climb in the world, the formula is decently accurate._

Disclaimer: I am not a sports coach, nutritionist or someone who specializes in these fields. I am a computer programmer, who was pretty obese + couldn't ride much and now I am very interested in learning about all this to become a better bike rider, be fit, and enjoy life.

Understanding the underlying science definitively gives me an edge so that I avoid the mistakes I used to make. For example, I used to stop at McDonalds to refuel. Yeah... don't judge me.

In [52]:
calories_burned = strava_calories_estimate(73.5, 46, 164.29, 34115, 4577, 134, 140, 17.3)
print(f"Calories burned using Strava calculation and average power for Alto de Letras: {calories_burned:.2f}")

Calories burned using Strava calculation and average power for Alto de Letras: 4776.10


2. Keytel Formula
The Keytel 2005 formula is a method used to estimate the number of calories burned during physical activity, particularly in running or walking. It was developed by Keytel et al. in 2005 as a way to improve upon earlier models by taking into account more variables such as heart rate, age, weight, gender, and the type of activity being performed.

The formula is as follows:

For men: 

![](./images/keytelmen.png)
​

For women: 

![](./images/keytelmen.png)
 

Where:

* HR is the heart rate in beats per minute
* Weight is the body weight in kilograms
* Age is the person's age in years
* T is the time of exercise in minutes
* 4.184 is the conversion factor to convert from kilocalories (kcal) to kilojoules (kJ)
* This formula is used in fitness trackers and other systems to provide more accurate calorie expenditure estimates during workouts.

Key Differences Between the Formulas:
* Heart Rate Coefficient: Men have a higher coefficient (0.6309) than women (0.4472), indicating that heart rate impacts calorie burn more for men.
* Weight Coefficient: Men also have a higher weight coefficient (0.1988) compared to women (0.1263).
* Age Coefficient: Age has a greater impact on men (0.2017) than on women (0.074).

These formulas adjust for individual factors like weight, heart rate, and age, making them more personalized for estimating calorie expenditure during physical activity.

In [44]:
def keytel_2005_cycling(gender, heart_rate, weight_kg, age, time_minutes):
    """
    Calculate calories burned during cycling using the Keytel 2005 formula.

    Parameters:
    gender (str): 'male' or 'female'
    heart_rate (float): Heart rate in beats per minute (bpm)
    weight_kg (float): Weight of the individual in kilograms
    age (int): Age of the individual in years
    time_minutes (float): Duration of exercise in minutes

    Returns:
    float: Estimated calories burned
    """
    if gender.lower() == 'male':
        calories_burned = (-55.0969 + (0.6309 * heart_rate) + (0.1988 * weight_kg) + (0.2017 * age)) * (time_minutes / 4.184)
    elif gender.lower() == 'female':
        calories_burned = (-20.4022 + (0.4472 * heart_rate) + (0.1263 * weight_kg) + (0.074 * age)) * (time_minutes / 4.184)
    else:
        raise ValueError("Gender must be either 'male' or 'female'")
    
    print(f"Calories burned: {calories_burned:.2f}")
    return calories_burned


X. Calculator.net

I used this online calculator to determine a rough estimate. 

![](./images/calculatordotnet.png)

Use this URL to get the calculation: [https://www.calculator.net/calories-burned-calculator.html?c2activity=b&c2speed=34.8&c2speedunit=kph&c2distance=121.83&c2distanceunit=k&c2weight=77&c2weightunit=k&ctype=2&x=Calculate#bydistance](https://www.calculator.net/calories-burned-calculator.html?c2activity=b&c2speed=34.8&c2speedunit=kph&c2distance=121.83&c2distanceunit=k&c2weight=77&c2weightunit=k&ctype=2&x=Calculate#bydistance)

In [45]:
calories_calculatordotnet = 4722

## Final Calculations

In [46]:
# Calculate the calories burned using the different formulas
calculated_calories_strava = 
calculated_calories_keytel = keytel_2005_cycling('male', average_heart_rate_bpm, weight_ticaban_kg, age, duration_moving_time_minutes)


SyntaxError: invalid syntax (3487549304.py, line 2)

In [ ]:
print("Calories Burned Calcuation")
print(f"Strava: {calories_strava}")

calories_burned = strava_calories_estimate(weight_ticaban_kg, age, distance_kilometers, duration_moving_time_seconds, elevation_gain_m, average_heart_rate_bpm, average_power_watts, average_speed_km_hour)


print("Keytel " + str(calories_keytel_chatgpt))
print("Calculator.net " + str(calories_keytel_copilot))
 